# Training a new CNN on the detected bounding boxes to distinguish between `helipad` and `false_positive`. 

# This notebook contained the entire training pipeline in order.

In [ ]:
import os
import numpy as np

import sys
sys.path.append('..')
sys.path.append('../../')

# Build the bounding box dataset

In [ ]:
from bb_build_dataset import BBBuildDataset

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat\\19\\"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\19\\"
model_number = 7
score_threshold = 0.0
iou_threshold = 0.5
output_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Detected_Boxes_TMS"
tms = True
groundtruth_bb = False
filter_categories = None

bb_build_dataset = BBBuildDataset(image_folder=image_folder,
                                  meta_folder=meta_folder,
                                  model_number=model_number,
                                  score_threshold=score_threshold,
                                  iou_threshold=iou_threshold,
                                  output_folder=output_folder,
                                  tms=tms,
                                  groundtruth_bb=groundtruth_bb,
                                  filter_categories=filter_categories)

bb_build_dataset.run()

# Annotate the Groundtruth

In [ ]:
from bb_dataset_groundtruth_TMS import BBDatasetGroundtruthTMS

image_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_zoomout5\\sat"
output_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_zoomout5_groundtruth\\"
source_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth\\"
start_index = 0

bb_build_groundtruth_tms = BBDatasetGroundtruthTMS(image_folder=image_folder,
                                                   output_folder=output_folder,
                                                   start_index=0,
                                                   source_from=source_folder)

bb_build_groundtruth_tms.run()

# Clean the groundtruth in case of mistakes

In [ ]:
from bb_dataset_cleaner import BBDatasetCleaner

image_folder = "C:\\Users\AISG\\Documents\\Jonas\\Detected_Boxes_3\\model_7_0.0\\test\\"
check_false_positive = True
start_index = 0

bb_dataset_cleaner = BBDatasetCleaner(image_folder=image_folder,
                                     check_false_positive=check_false_positive,
                                     start_index=start_index)

bb_dataset_cleaner.run()

# Split the dataset between train and test set

In [ ]:
from bb_dataset_groundtruth_tms_split_test_train import BBDatasetGroundtruthTMSSplitTestTrain

image_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth\\"
output_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth_split\\"
test_size = 0.2

bb_dataset_groundtruth_tms_split_test_train = BBDatasetGroundtruthTMSSplitTestTrain(image_folder=image_folder,
                                                                                   output_folder=output_folder,
                                                                                   test_size=test_size)

bb_dataset_groundtruth_tms_split_test_train.run()

# Now that the dataset is ready for training, train a new CNN on it to distinguish between `helipad` and `false_positive`

In [ ]:
from bb_training_manager import BBTrainingManager

image_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth_split\\"
model_filename = "bb_model_manilla_adam_64.h5"

bbtraining_manager = BBTrainingManager(image_folder)

# run the training
bbtraining_manager.run()
# evaluate accuracy on the test set
bbtraining_manager.evaluate()
# save the model
bbtraining_manager.save(filename=model_filename)
# plot the metrics curves
bbtraining_manager.plot()

# Predict the class of new bounding boxes using the new CNN

In [ ]:
from bb_predict import BBPredict

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat\\"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 10
model_path = "final_model.h5"
tms = True

index_path = "../helipad_path_over_0_m10.txt"

bbpredict = BBPredict(image_folder=image_folder,
                      meta_folder=meta_folder,
                      model_number=model_number,
                      model_path=model_path,
                      tms=tms,
                      index_path=index_path)

bbpredict.run()

# Build a Placemark files taking into account the prediction on the bounding boxes

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../detection')
from detection.build_placemarks import BuildPlacemarks


meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 10
threshold = 0.7

index_path = "helipad_path_over_0_m10.txt"
# index_path = None

knn = True
model_name = "cnn_validation"
model_validation_threshold = 0.7

prefix = "Manilla_"

build_placemarks = BuildPlacemarks(meta_folder,
                                   model_number,
                                   threshold,
                                   knn=knn,
                                   model_name=model_name,
                                   model_validation_threshold=model_validation_threshold,
                                   index_path=index_path, 
                                   prefix=prefix)

build_placemarks.run()